<a href="https://colab.research.google.com/github/FaragoFa/Materia-de-aprendizaje/blob/master/ROC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importo librerias

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import plt
from numpy import random
import pandas as pd
import numpy as np

Genero y acomodo el dataset

In [ ]:
x = random.normal(loc=0, scale=1, size=50) #Creo dataset de 50 mandarinas
y = random.normal(loc=0, scale=1, size=50) #Creo dataset de 50 naranjas (target)

#Transpongo la fila y creo una columna nueva que otorgue 1 a todas las naranjas
dfy = pd.DataFrame([y])
dfy = dfy.transpose()
dfy=dfy.assign(Naranjas = 1)
dfy.columns = ['Tamaño', 'Naranjas']

#Transpongo la fila y creo una columna nueva que otorgue 0 a todas las mandarinas
dfx = pd.DataFrame([x])
dfx = dfx.transpose()
dfx=dfx.assign(Naranjas = 0)
dfx.columns = ['Tamaño', 'Naranjas']

#Uno mandarinas con naranjas
df = pd.concat([dfx, dfy])

#data_X, class_label = make_classification(n_samples=1000, n_classes=2, weights=[1,1], random_state=1)

Defino la funcion para plotear la curva ROC

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color="orange", label="ROC")
    plt.plot([0, 1], [0, 1], color="darkblue", linestyle="--")
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()

Divido el dataset en train y test

In [ ]:
trainX, testX, trainy, testy = train_test_split(df['Tamaño'], df['Naranjas'], test_size=0.3, random_state=1)

Entreno a mi modelo

In [ ]:
model = LogisticRegression()
model.fit(trainX, trainy)

Predigo probabilidades para los datos de prueba, Manteniendo solo las probabilidades de la clase positiva

In [ ]:
probs = model.predict_proba(testX)
probs = probs[:, 1]

ValueError: ignored

Calculo la puntuación AUC

In [ ]:
auc = roc_auc_score(testy, probs)
auc

Obtengo y grafico curva ROC

In [ ]:
fpr, tpr, thresholds = roc_curve(testy, probs)
plot_roc_curve(fpr, tpr)